In [38]:
import json
import pandas as pd
import numpy as np
from time import time
from glob import glob

In [39]:
pd.__version__, np.__version__

('1.4.1', '1.21.5')

In [40]:
# abs path -> 22-03-07/ *.json
best = glob('22-03-07/best*.json')
item = glob('22-03-07/item*.json')
products = glob('22-03-07/products*.json')
#category1 == category2 = True
category = glob('22-03-07/category1.json')

In [41]:
best, item, products, category

(['22-03-07/bestshots2.json', '22-03-07/bestshots1.json'],
 ['22-03-07/items1.json', '22-03-07/items2.json'],
 ['22-03-07/products2.json', '22-03-07/products1.json'],
 ['22-03-07/category1.json'])

In [42]:
import pandas as pd
import json

class Preprocess:
    
    def __init__(self, best, item, products, category, style_ths=0.1):
        """
        style_ths : top_style_predictions 에서 범위 조절 (e.g. 0.1 이상으로 예측 점수를 받은 스타일만 받아옴 - maximum 3개)
        """
        self.style_ths = style_ths
        self.style_ths_name = str(style_ths)[-1]
        
        self.best = best
        self.item = item
        self.products = products
        self.category = category
        
        self.best_json = self.read_json(best)
        self.item_json = self.read_json(item)
        self.products_json = self.read_json(products).drop_duplicates('_id') # products json duplicated deleted
        self.category_json = self.read_json2(category).reset_index(drop=True)
        self.best_item, self.category, self.products_4, self.products_b = self.preprocess(self.best_json, 
                                                                           self.item_json, 
                                                                           self.products_json, 
                                                                           self.category_json)

    def read_json(self,json_file):
        df = pd.DataFrame()
        if json_file != self.products:  
            for file in json_file:
                x = pd.read_json(file)
                df = pd.concat([df, x])
        else:
            # why -> products json duplicated 
            for file in json_file:
                x = pd.read_json(file).T.reset_index(drop=True)    
                df = pd.concat([df, x])
        return df

    def read_json2(self,category):
        for i in category:
            with open(i) as js:
                json_data = json.load(js)
        cat_4, cat_b = pd.DataFrame(json_data['421B6D0E746C4E6D']), pd.DataFrame(json_data['B57D4F97C0E44A11'])
        category = pd.concat([cat_4, cat_b])
        return category
    
    def get_productId(self, items_list):
        new_list = []
        for item in items_list:
            new_list.append(item['productId'])
        return new_list
    
    # category['children'] -> name(label) 
    def find_category_df(self, category):
        df = pd.DataFrame()
        for i in category['children']:
            df2 = pd.DataFrame(i)
            df = pd.concat([df, df2])
        return df
    
    # 4/b 분리하는 함수 => best, item, products, category
    # return => best, item, products, category / best, item, products, category
    
    def preprocess(self, best, item, products, category):
        
        item.rename(columns={'enterpriseId':'enterprise_id', '_id':'id'}, inplace=True)
        templates = pd.merge(best, item, on=['enterprise_id', 'projectId', 'id'])
        
        # 2 enterprise id 
        ent2, ent1 = templates['enterprise_id'].unique().tolist()
        
        # get item_id 
        templates['items'] = templates['items'].apply(self.get_productId)
        
        # find_category_id_preprocess
    
        # edit new columns style_name, style_score > 0.1 
        templates[f'top_style_{self.style_ths_name}'] = templates['style_predictions'].apply(
            lambda x: sorted(
                [(name, score) for name, score in x.items() if score > self.style_ths],
                key=lambda x: x[1],
                reverse=True)[:3]
        )
        # del list, style score
        templates[f'top_style_{self.style_ths_name}'] = templates[f'top_style_{self.style_ths_name}'].apply(
            lambda x: ' '.join([name for name, score in x]))
        
        # Edit best_item['items'] = list(values) -> values
        items_stack = pd.DataFrame(templates['items'].apply(lambda x: pd.Series(x)).stack()).reset_index(1, drop=True) 
        products_df = pd.merge(templates[['enterprise_id', 
                                          f'top_style_{self.style_ths_name}', 
                                          'top_style', 
                                          'projectId', 
                                          'awesome_score']].reset_index(), 
                               items_stack.reset_index(), 
                               on='index').drop(['index'], axis=1).rename(columns = {0:'product_id'})
        prod_tags_df = products[['_id', 'tags', 'name', 'images', 'categories']]
        products_df = pd.merge(products_df, 
                               prod_tags_df, 
                               left_on='product_id', 
                               right_on='_id').drop(['_id'], axis=1)
        products_df = pd.merge(products_df, 
                               products_df['product_id'].value_counts().reset_index(),
                               left_on='product_id',
                               right_on='index').rename(columns = {'product_id_x': 'product_id',
                                                                   'product_id_y':'use_count'}).drop(['index'], axis=1)
        
        # category[categories] = list(values) -> values
#         products_df['category_name'] = products_df['categories'].apply(lambda x: pd.Series(x)).reset_index(drop=True).drop(columns=1)
        products_df['category_name'] = products_df['categories'].apply(lambda x: x[0] if len(x) > 0 else np.nan)
        
        
        ## new code
        # find_category_id_preprocess
        cat_a = self.find_category_df(self.category_json)
        cat_a = pd.merge(self.category_json[['_id', 'name']], 
                         cat_a[['name', '_id', 'enterpriseId', 'parentId', 'children']], 
                         left_on='_id', right_on='parentId').drop(['parentId'], axis=1)
        
        cat_b = self.find_category_df(cat_a)
        cat_b = pd.merge(self.category_json[['_id', 'name']], 
                 cat_b[['name', 'enterpriseId', 'parentId', 'grandparentId', '_id']], 
                 left_on='_id', 
                 right_on='grandparentId').drop(['grandparentId'], axis=1)
        
        self.products_json['category_name'] = self.products_json['categories'].apply(lambda x: x[0] if len(x) > 0 else np.nan)
#         self.products_json['category_name'] = self.products_json['categories'].apply(lambda x: pd.Series(x)).reset_index(drop=True).drop(columns=1)
        
        # category_df['name_x'] = category['children']
        # category_df['name_y'] = category['children'][children]
        prod_a = pd.merge(self.products_json[['_id', 'name', 'category_name']], 
                         cat_a[['name_x', 'name_y', '_id_y', 'enterpriseId', 'children']], 
                         left_on='category_name', right_on='_id_y', how='left').drop(['_id_y'], axis=1)
        
        prod_b = pd.merge(self.products_json[['_id', 'name', 'category_name']], 
                         cat_b[['name_x', 'name_y', '_id_y', 'enterpriseId']], 
                         left_on='category_name', right_on='_id_y', how='left').drop(['_id_y'], axis=1)
        
        category_df = pd.merge(prod_a, 
                         prod_b[['_id', 'category_name', 'name_x', 'name_y']].rename(columns={'name_y':'name_z'}),
                         on='_id'
                        )
        category_df['name_x_x'] = category_df['name_x_x'].fillna(category_df['name_x_y'])
        category_df = category_df.drop(['name_x_y', 'category_name_x', 'category_name_y','children'], axis=1)
        category_df.columns = ['product_id', 'name', 
                            'name_x', 'name_y', 
                            'enterpriseId', 'name_z']
        category_df = category_df[['product_id', 'name', 
                                'name_x', 'name_y', 'name_z', 
                                'enterpriseId']]
        
        category_df = pd.merge(self.products_json[['_id', 'name', 'tags', 'dimensions', 'images']], 
                               category_df[['product_id', 'name_x', 'name_y', 'name_z']], 
                                left_on = '_id',
                                right_on = 'product_id')
        
        products_df = pd.merge(products_df[['enterprise_id', 'projectId', 'top_style_1', 
                                            'top_style', 'awesome_score', 'product_id', 'use_count']], 
                               category_df, 
                               on='product_id')
        
        products_df_4 = products_df[products_df['enterprise_id'] == ent1].reset_index(drop=True)
        products_df_b = products_df[products_df['enterprise_id'] == ent2].reset_index(drop=True)
        
        return templates, category_df, products_df_4, products_df_b

In [43]:
start_time = time()
data = Preprocess(best, item, products, category)
print(time() - start_time)

/var/folders/9_/rnj5svk11p1157tkdv28yb6r0000gn/T/ipykernel_49011/1001001398.py:90: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  items_stack = pd.DataFrame(templates['items'].apply(lambda x: pd.Series(x)).stack()).reset_index(1, drop=True)


2.175428867340088


In [44]:
data.products_4.iloc[398]

enterprise_id                                     421B6D0E746C4E6D
projectId                                  X8_sB1nD0A90AED920B4963
top_style_1                    NATURAL MINIMAL CLASSIC_AND_ANTIQUE
top_style                                                  NATURAL
awesome_score                                             0.887253
product_id                                 Xho2Q_R7168F63EA0DC4AFC
use_count                                                        8
_id                                        Xho2Q_R7168F63EA0DC4AFC
name                                              대형 아레카야자 ( 화이트 )
tags                                                            []
dimensions       {'width': 1186, 'height': 2104, 'depth': 1452,...
images           [https://resources.archisketch.com/product/Xho...
name_x                                                       Décor
name_y                                            Flowers & Plants
name_z                                                        

In [46]:
data.products_json['enterpriseId'].unique()

array(['B57D4F97C0E44A11', '421B6D0E746C4E6D', 'X2y05bCA998108E75E74C7D',
       '15BEED4FED5A48FD', 'B6C8251995CA4867', 'XoICUVPB5ECF48BA60E4DED',
       'X0IUXyjAACE7265029649CB', 'XQACQJ_1585CF9DCDF24738',
       'Xt0h3oh69E61A40DB78408C', 'XRXsOll73EE43188FC34DB9',
       'XpVbS6q64E4F3457D0643A2', 'Xj4P1Ah517BE5A5AEE84670',
       'XTkoXgbF2D6830243D64FAD'], dtype=object)

In [ ]:
x.category

In [ ]:
x.prducts_4

In [ ]:
len(x.products_4)

In [ ]:
def find_category_df(category):
    df = pd.DataFrame()
    for i in category['children']:
        df2 = pd.DataFrame(i)
        df = pd.concat([df, df2])
    return df

df = find_category_df(x.category_json)
df2 = find_category_df(df).drop('parentId', axis=1).rename(columns={'grandparentId':'parentId'})
# dd = pd.concat([df, df2], ignore_index=True)
# dd = pd.merge(x.category[['name', '_id']], dd, left_on='_id', right_on='parentId').rename(columns={'_id_y' : '_id'}).drop(columns=['_id_x'])

In [ ]:
df2

In [ ]:
df

In [ ]:
x.category_json

In [ ]:
df2

In [ ]:
x.products_json['category_name'] = x.products_json['categories'].apply(lambda x: pd.Series(x)).reset_index(drop=True).drop(columns=1)
# categories_name.apply(find_category)

In [9]:
prod_a = pd.merge(x.products_json[['_id', 'name', 'category_name']], 
         df[['name', '_id', 'enterpriseId']], 
         left_on='category_name', right_on='_id', how='left')

prod_b = pd.merge(x.products_json[['_id', 'name', 'category_name']], 
         df2[['name', '_id', 'enterpriseId']], 
         left_on='category_name', right_on='_id', how='left').sort_values(by='_id_x')


NameError: name 'x' is not defined

In [10]:
prod_a.head()

NameError: name 'prod_a' is not defined

In [11]:
prod_b.head()

NameError: name 'prod_b' is not defined

In [12]:
category = pd.concat([prod_a.rename(columns={'name_y':'category_a'}), 
           prod_b[['name_y']].rename(columns={'name_y':'category_b'})], axis=1)

NameError: name 'prod_a' is not defined

In [13]:
category

['22-03-07/category1.json']

In [14]:
category.head()

AttributeError: 'list' object has no attribute 'head'

In [15]:
prod = x.products_json

NameError: name 'x' is not defined

In [16]:
prod.info()

NameError: name 'prod' is not defined

In [17]:
_id, name, enterprisedId, tags, dimensions, images

NameError: name '_id' is not defined

In [18]:
x = pd.merge(prod[['_id', 'name', 'enterpriseId', 'tags', 'dimensions', 'images']], category[['_id_x', 'name_x', 'category_a', 'enterpriseId', 'category_b']], left_on ='_id', right_on = '_id_x')

NameError: name 'prod' is not defined

In [19]:
x

NameError: name 'x' is not defined

In [20]:
x.products_4.head(10)

NameError: name 'x' is not defined

In [47]:
templates = data.best_item
len(templates)

4556

In [48]:
templates.head(2)

,awesome_score,date,datetime,enterprise_id,id,projectId,style_predictions,top_score,top_style,remainTime,...,background,tags,roomTag,watermark,comment,items,archive,createdAt,updatedAt,top_style_1
0,0.993074,2022-03-06,2022-03-06 04:49:13.054,B57D4F97C0E44A11,X9dj2Ve97CAE025F82F4A8B,X9djPepD57A2BDCD418437C,"{'BOHEMIAN': 0.00019316500000000003, 'CLASSIC_...",0.986163,NATURAL,00:01:41.31,...,5,"[렌지대, 리바트, 침대형소파, 가공목(mdf외), 4층, 식탁의자, 아이와비, o...",0,{'image': 'https://resources.archisketch.com/e...,"{'title': '', 'body': ''}","[XjDwfV4C2377DB2C32B4EBD, XrTVBz466807713843D4...",NaN,2022-03-06T04:49:13.054Z,2022-03-06T04:51:24.255Z,NATURAL
1,0.990285,2022-03-06,2022-03-06 08:11:26.221,B57D4F97C0E44A11,X9eSIiNE9BF42316EF94426,X9dban93BD94E4045124D77,"{'BOHEMIAN': 0.000295741, 'CLASSIC_AND_ANTIQUE...",0.939667,MODERN,00:01:40.95,...,8,[],4,{'image': 'https://resources.archisketch.com/e...,"{'title': '', 'body': ''}","[AA4A41116A904BFC, F1BBC6E62B02416E, 88FFE5485...",NaN,2022-03-06T08:11:26.221Z,2022-03-06T08:30:22.868Z,MODERN


In [49]:
items_stack = pd.DataFrame(templates['items'].apply(lambda x: pd.Series(x)).stack()).reset_index(1, drop=True) 
items_stack

/var/folders/9_/rnj5svk11p1157tkdv28yb6r0000gn/T/ipykernel_49011/894167859.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  items_stack = pd.DataFrame(templates['items'].apply(lambda x: pd.Series(x)).stack()).reset_index(1, drop=True)


,0
0,XjDwfV4C2377DB2C32B4EBD
0,XrTVBz466807713843D41CB
0,1DFB69B26D714ED5
0,68787D99716C4340
0,XZvSCSS8CD1831D050045BB
...,...
4555,XxzIVRTC57DDD7111544A95
4555,XPZDgbs0900F2C2D8F64A49
4555,63515C424CA0489D
4555,XkcvCQRD4A4AF021FE54BFD


In [50]:
items_stack.duplicated().sum(), 54939-45411

(45411, 9528)

In [51]:
products_df = pd.merge(templates[['enterprise_id', 
                                  'top_style_1', 
                                  'top_style', 
                                  'projectId', 
                                  'awesome_score']].reset_index(), 
                       items_stack.reset_index(), 
                       on='index').drop(['index'], axis=1).rename(columns = {0:'product_id'})
len(products_df)

54939

In [52]:
products_df[products_df['product_id'] == 'XYmsm7pDACA37C0FCB843C3']

,enterprise_id,top_style_1,top_style,projectId,awesome_score,product_id
11514,421B6D0E746C4E6D,MODERN,MODERN,X8lxCwVE40C012658D14CAF,0.990975,XYmsm7pDACA37C0FCB843C3
11530,421B6D0E746C4E6D,MODERN,MODERN,X8lxCwVE40C012658D14CAF,0.990975,XYmsm7pDACA37C0FCB843C3
11546,421B6D0E746C4E6D,MODERN,MODERN,X8lxCwVE40C012658D14CAF,0.990975,XYmsm7pDACA37C0FCB843C3
11562,421B6D0E746C4E6D,MODERN,MODERN,X8lxCwVE40C012658D14CAF,0.990975,XYmsm7pDACA37C0FCB843C3


In [37]:
products_df.drop_duplicates()

,enterprise_id,top_style_1,top_style,projectId,awesome_score,product_id
0,B57D4F97C0E44A11,NATURAL,NATURAL,X9djPepD57A2BDCD418437C,0.993074,XjDwfV4C2377DB2C32B4EBD
1,B57D4F97C0E44A11,NATURAL,NATURAL,X9djPepD57A2BDCD418437C,0.993074,XrTVBz466807713843D41CB
2,B57D4F97C0E44A11,NATURAL,NATURAL,X9djPepD57A2BDCD418437C,0.993074,1DFB69B26D714ED5
3,B57D4F97C0E44A11,NATURAL,NATURAL,X9djPepD57A2BDCD418437C,0.993074,68787D99716C4340
4,B57D4F97C0E44A11,NATURAL,NATURAL,X9djPepD57A2BDCD418437C,0.993074,XZvSCSS8CD1831D050045BB
...,...,...,...,...,...,...
54934,B57D4F97C0E44A11,MODERN,MODERN,X3G2GQG755739AB6AE6445F,0.804627,XxzIVRTC57DDD7111544A95
54935,B57D4F97C0E44A11,MODERN,MODERN,X3G2GQG755739AB6AE6445F,0.804627,XPZDgbs0900F2C2D8F64A49
54936,B57D4F97C0E44A11,MODERN,MODERN,X3G2GQG755739AB6AE6445F,0.804627,63515C424CA0489D
54937,B57D4F97C0E44A11,MODERN,MODERN,X3G2GQG755739AB6AE6445F,0.804627,XkcvCQRD4A4AF021FE54BFD
